In [8]:
%pip install matplotlib pandas prettytable

  Using cached prettytable-3.9.0-py3-none-any.whl (27 kB)

[notice] A new release of pip available: 22.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [32]:
# Import data from CSV into a panda dataframe from ./extract-2024-01-23T12_29_42.992Z.csv   

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from datetime import datetime
from datetime import timedelta
import os
import sys

# Read CSV file into a panda dataframe
df = pd.read_csv('./extract-2024-01-23T12_29_42.992Z.csv', header=0, sep=',', parse_dates=[0], index_col=0)


# print number of rows and columns
print(df.shape)

# print the column names
print(df.columns)


(33377, 6)
Index(['Service', 'Resource', 'Duration', 'Http Method', 'Status Code',
       'active_record.instantiation.record_count'],
      dtype='object')


In [41]:


# get rid of the double quotes first
df['Duration_float'] = df['Duration'].str.replace('"', '')

#then get rid of the trailing 'ms' or 's' and multiply by 1000 those wich ended to 's' to convert to milliseconds
df['Duration_float'] = df['Duration_float'].str.replace('ms', '')
df['Duration_float'] = df['Duration_float'].str.replace('s', '000')

# convert Duration to float
df['Duration_float'] = df['Duration_float'].astype(float)

# group by Resource, sum the Duration and calculate the average active_record.instantiation.record_count
df_groupby = df.groupby('Resource').agg({'Duration_float': 'sum', 'active_record.instantiation.record_count': 'mean'}).sort_values(by=['Duration_float'], ascending=False)


#df_groupby = df.groupby('Resource').sum().sort_values(by=['Duration_float'], ascending=False)

# print df_groupby with prettytable
# convert Duration to human readable format

from prettytable import PrettyTable
x = PrettyTable()
x.field_names = ["Resource", "Duration", "Record count"]
for index, row in df_groupby.iterrows():
    if row['Duration_float'] < 10000:
        break
    x.add_row([index, timedelta(milliseconds=row['Duration_float']), row['active_record.instantiation.record_count']])
print(x)




+---------------------------------------+----------------+--------------------+
|                Resource               |    Duration    |    Record count    |
+---------------------------------------+----------------+--------------------+
|           "GroupProjectRole"          | 0:32:18.348800 | 53457.32551020408  |
|         "EntityResponsibility"        | 0:22:55.798110 | 51118.686440677964 |
| "CustomProjectPlanningAttributeValue" | 0:20:16.159170 | 13960.707022834986 |
|               "Signoff"               | 0:15:14.728880 | 68248.12162162163  |
|     "CustomControlAttributeValue"     | 0:13:45.311350 | 42503.359628770304 |
|               "Control"               | 0:11:47.960760 | 84361.17026748971  |
|             "ProjectRole"             | 0:10:45.424460 | 543950.7462028418  |
|               "FlexDate"              | 0:10:23.401680 | 100471.67842822775 |
|              "Objective"              | 0:08:57.982200 | 25771.71602126044  |
|          "ProcessWalkthrough"         